In [ ]:
import xarray as xr
import pandas as pd
import fsspec
import dask

dask.config.set({"distributed.logging.distributed": "critical"})

In [ ]:
# from dask.distributed import Client
# setup a local dask cluster
# client = Client()
# client

from dask_gateway import Gateway

gateway = Gateway()
options = gateway.cluster_options()
options.worker_cores = 4
options.worker_memory = 16
cluster = gateway.new_cluster(cluster_options=options)
cluster.adapt(minimum=1, maximum=40)
cluster

In [ ]:
client = cluster.get_client()
client

In [ ]:
bucket = "gs://carbonplan-data/raw/terraclimate/4000m/raster.zarr"
mapper = fsspec.get_mapper(bucket)

In [ ]:
%%time
ds = xr.open_zarr(mapper, consolidated=True)

In [ ]:
ds

In [ ]:
ds.dims

In [ ]:
null_count = ds.isnull().mean("time")
null_count = null_count.where(null_count != 1)

In [ ]:
pdsi = null_count.pdsi.load()

# [::50, ::50].plot()

In [ ]:
pdsi.coarsen(lat=2, lon=2).mean().plot(robust=True)
# pdsi

In [ ]:
df = pd.read_csv("FIA_forBill_DroughtRiskV1_08182020.csv")  # .iloc[0:10000]
df.head()

In [ ]:
import math

n = 5

size = math.ceil(len(df) / n)
assert len(df) <= size * n
print(size)

In [ ]:
index_ds = df[["lat", "lon"]].to_xarray()
index_ds

In [ ]:
# %%time
# ds_cond = ds.sel(lat=index_ds['lat'], lon=index_ds['lon'], method='nearest')
# ds_cond

In [ ]:
from tqdm import tqdm

In [ ]:
ds_list = []
for k, ids in tqdm(index_ds.groupby(index_ds.index // size)):
    ds_list.append(ds.sel(lat=ids["lat"], lon=ids["lon"], method="nearest"))

In [ ]:
# ds_list[:2]

In [ ]:
sum(d.nbytes for d in ds_list) / 1e9

In [ ]:
parts = []
for d in tqdm(ds_list):
    part = d.compute()
    parts.append(part)

In [ ]:
ds_cond = xr.concat(parts, dim="index")
# ds_cond = ds_cond.chunk({'index': 40000, 'time': 240})

In [ ]:
ds_cond.PDSI.isnull().sum("index").plot()

In [ ]:
ds_cond = xr.concat(parts, dim="index")
ds_cond = ds_cond.chunk({"index": 40000, "time": 240})

In [ ]:
from dask.distributed import Client

client = Client()
client

In [ ]:
bucket = "gs://carbonplan-scratch/terraclimate-fia-cond.zarr"
mapper2 = fsspec.get_mapper(bucket)

ds_cond.to_zarr(mapper2, mode="w")

In [ ]:
# bucket = "gs://carbonplan-scratch/terraclimate-fia-cond.zarr"
# mapper2 = fsspec.get_mapper(bucket)

# ds_cond = xr.open_zarr(mapper2)

In [ ]:
# ds_cond_ann = ds_cond.resample(time='AS').mean()
# ds_cond_ann

In [ ]:
def weighted_mean(ds, *args, **kwargs):
    weights = ds.time.dt.days_in_month
    return ds.weighted(weights).mean(dim="time")


ds_cond_ann = ds_cond.resample(time="AS").map(weighted_mean, dim="time")

In [ ]:
ds_cond_ann = ds_cond_ann.chunk({"index": -1, "time": -1})
ds_cond_ann["lon"] = ds_cond_ann["lon"].load()
ds_cond_ann["lat"] = ds_cond_ann["lat"].load()
ds_cond_ann

In [ ]:
# from zarr.storage import ZipStore

# store = ZipStore('terraclimate-fia-cond-ann.zarr.zip', mode='w')

In [ ]:
encoding = {k: {} for k in ds_cond_ann.data_vars}
encoding

In [ ]:
bucket = "gs://carbonplan-scratch/terraclimate-fia-cond-ann-3.zarr"
mapper3 = fsspec.get_mapper(bucket, create=True)

from dask.diagnostics import ProgressBar

with ProgressBar():
    ds_cond_ann.to_zarr(mapper3, mode="w", consolidated=True, encoding=encoding)

In [ ]:
da = ds_cond_ann.PDSI.load()

In [ ]:
da.isel(index=slice(20000))

In [ ]:
da.isnull().sum("index").plot()